In [1]:
import pandas as pd
import numpy as np
import re
import string
import emoji
import nltk

In [2]:
def pre_processing(text):
    if isinstance(text, str):
        
        # Removing URLs
        def remove_url(text):
            pattern = re.compile(r'https?://\S+|www\.\S+')
            return pattern.sub(r'', text)

        text = remove_url(text)

        # Removing HTML tags
        def remove_html_tags(text):
            pattern = re.compile('<.*?>')
            return pattern.sub(r'', text)

        text = remove_html_tags(text)
        
        # Removing Emojis
        def remove_emojis(text):
            emoji_unicode_dict = emoji.get_emoji_unicode_dict(lang='en')
            emoji_pattern = re.compile("|".join(emoji_unicode_dict.keys()))
            return emoji_pattern.sub(r'', text)
    
        text = remove_emojis(text)

    return text

In [3]:
def create_passages(text, max_words_per_passage=100):
    if isinstance(text, str):
        sentences = nltk.sent_tokenize(text)
        passages = []
        passage = ""
        word_count = 0

        for sentence in sentences:
            words = nltk.word_tokenize(sentence)
            word_count += len(words)

            if word_count <= max_words_per_passage:
                passage += " " + sentence
            else:
                passages.append(passage.strip())
                passage = sentence
                word_count = len(words)

        if passage:
            passages.append(passage.strip())

        return passages

### Emails Data

In [4]:
emails_data = pd.read_csv('Email/emails_raw.csv')

#filtering 2022 emails
emails_2022 = emails_data[emails_data['Year'] == 2022]
emails_2022.shape

(80100, 13)

In [5]:
emails_df = emails_2022[['Name', 'Subject','Body']]
emails_df.head()

,Name,Subject,Body
34715,DUE @ 11:59 PM | Upset the Setup,re: President Joe Biden,☆ OFFICIAL UTS APPROVAL POLL | REGISTERED TO P...
34863,Coach Tuberville,🎉 Happy New Year 🎉,"96 Peter, The countdown is on – 2022 is quickl..."
34872,Derek Kilmer for Congress,One last update,"Peter, Before tonight's festivities begin, we ..."
34873,Sarah Longwell,A great year for democracy,We thought you might want to see this special ...
34875,Donald J. Trump,2022 is almost over,--- | | | | | | | Untitled Document![Trump 202...


In [6]:
# before pre-processing

emails_df['Body'][34875:34900]

/var/folders/kg/_lk6hlqn05z_ghjpvh5y1z400000gn/T/ipykernel_41998/2943005930.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  emails_df['Body'][34875:34900]


70194    Leora Levy for U.S. Senate I'm beyond honored ...
70195    Friend – I'm less than 24 hours away from my f...
70196    I’m going to tell you about my tattoos. | | |!...
70197    Can I count on you? https://electalexis.us4.li...
70198    Peter — in June, the Supreme Court OVERTURNED ...
70199    If you believe you received this message in er...
70200    https://www.thefairnessproject.org/ [https://w...
70201    Folks, I have never felt more hopeful about th...
70202    JD Vance for US Senate                        ...
70203    [1]Charles Booker Peter, take a look at this: ...
70204    Herschel Walker for Senate This is bad, Peter....
70205    Did you see the news, Peter? MAGA Republican M...
70206    Derek, Marriage equality is the law of the lan...
70207    Here's the thing: People like being able to ch...
70208    Tyler Kistner for Congress Building up our fie...
70209    Official RNC Updates The raid on Pres. Trump’s...
70210    Feenstra for Congress Delete once read        .

In [7]:
# pre-processing on "Body" column

emails_df['Body'] = emails_df['Body'].apply(pre_processing)

/var/folders/kg/_lk6hlqn05z_ghjpvh5y1z400000gn/T/ipykernel_41998/582073124.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emails_df['Body'] = emails_df['Body'].apply(pre_processing)


In [8]:
# after pre-processing

emails_df['Body'][34875:34900]

/var/folders/kg/_lk6hlqn05z_ghjpvh5y1z400000gn/T/ipykernel_41998/1797652354.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  emails_df['Body'][34875:34900]


70194    Leora Levy for U.S. Senate I'm beyond honored ...
70195    Friend – I'm less than 24 hours away from my f...
70196    I’m going to tell you about my tattoos. | | |!...
70197    Can I count on you?  The end of August is fast...
70198    Peter — in June, the Supreme Court OVERTURNED ...
70199    If you believe you received this message in er...
70200     [ Team— In the first major test of the 2022 e...
70201    Folks, I have never felt more hopeful about th...
70202    JD Vance for US Senate                        ...
70203    [1]Charles Booker Peter, take a look at this: ...
70204    Herschel Walker for Senate This is bad, Peter....
70205    Did you see the news, Peter? MAGA Republican M...
70206    Derek, Marriage equality is the law of the lan...
70207    Here's the thing: People like being able to ch...
70208    Tyler Kistner for Congress Building up our fie...
70209    Official RNC Updates The raid on Pres. Trump’s...
70210    Feenstra for Congress Delete once read        .

In [9]:
# j = 0
# for i in emails_df['Body']:
#     if j <=10:
#         print(i)
#         print("**********************************")
#         print()
#     j +=1

In [10]:
emails_df['Passages'] = emails_df['Body'].apply(create_passages)

/var/folders/kg/_lk6hlqn05z_ghjpvh5y1z400000gn/T/ipykernel_41998/4239503947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emails_df['Passages'] = emails_df['Body'].apply(create_passages)


In [11]:
emails_df

,Name,Subject,Body,Passages
34715,DUE @ 11:59 PM | Upset the Setup,re: President Joe Biden,☆ OFFICIAL UTS APPROVAL POLL | REGISTERED TO P...,[☆ OFFICIAL UTS APPROVAL POLL | REGISTERED TO ...
34863,Coach Tuberville,🎉 Happy New Year 🎉,"96 Peter, The countdown is on – 2022 is quickl...","[96 Peter, The countdown is on – 2022 is quick..."
34872,Derek Kilmer for Congress,One last update,"Peter, Before tonight's festivities begin, we ...","[Peter, Before tonight's festivities begin, we..."
34873,Sarah Longwell,A great year for democracy,We thought you might want to see this special ...,[We thought you might want to see this special...
34875,Donald J. Trump,2022 is almost over,--- | | | | | | | Untitled Document![Trump 202...,[--- | | | | | | | Untitled Document! [Trump 2...
...,...,...,...,...
186607,Justices Gorsuch and Kavanaugh LIED to the Ame...,5X match to hold Republicans accountable,"Team, Representatives Ted Lieu and Alexandria ...","[Team, Representatives Ted Lieu and Alexandria..."
186608,Dan Goldman,A new playbook in Congress,"Today marks 50 days until Election Day, Peter....","[Today marks 50 days until Election Day, Peter..."
186609,Team Beshear,Andy's opponents are threatening to undo our p...,FIRST: It was reported that former disgraced g...,[FIRST: It was reported that former disgraced ...
186611,Tom Malinowski,thank you for keeping up this fight,͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ...,"[, ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏..."


In [12]:
emails_df.to_csv('Email/emails_processed.csv')

### Podcast Data

In [13]:
podcast_data = pd.read_csv('Podcast/podcast_data_raw.csv')
podcast_data

,Podcast,Date,Part,Transcript
0,Conservative_Review_with_Daniel_Horowitz,2022-05-12,1,And welcome back fellow American patriots and...
1,Conservative_Review_with_Daniel_Horowitz,2022-11-04,1,And welcome back fellow American patriots and...
2,Conservative_Review_with_Daniel_Horowitz,2022-02-10,1,"Set five years after Jedi Fallen Order, follo..."
3,Conservative_Review_with_Daniel_Horowitz,2022-05-16,1,Politics Without the Soap Opera with unfilter...
4,Conservative_Review_with_Daniel_Horowitz,2022-09-13,1,And welcome back fellow American patriots and...
...,...,...,...,...
6944,Louder_with_Crowder,2022-11-10,1,"Alright, Mr. Jeremy Quartering. Does your aud..."
6945,Louder_with_Crowder,2022-09-06,14,"You're running for press, you socialist cheat..."
6946,Louder_with_Crowder,2022-09-06,1,"Yeah, uh, don't get it twisted This late nigh..."
6947,Louder_with_Crowder,2022-09-07,2,It's Parody Week! You can find today's track ...


In [14]:
podcast_data['Transcript'] = podcast_data['Transcript'].apply(pre_processing)
podcast_data

,Podcast,Date,Part,Transcript
0,Conservative_Review_with_Daniel_Horowitz,2022-05-12,1,And welcome back fellow American patriots and...
1,Conservative_Review_with_Daniel_Horowitz,2022-11-04,1,And welcome back fellow American patriots and...
2,Conservative_Review_with_Daniel_Horowitz,2022-02-10,1,"Set five years after Jedi Fallen Order, follo..."
3,Conservative_Review_with_Daniel_Horowitz,2022-05-16,1,Politics Without the Soap Opera with unfilter...
4,Conservative_Review_with_Daniel_Horowitz,2022-09-13,1,And welcome back fellow American patriots and...
...,...,...,...,...
6944,Louder_with_Crowder,2022-11-10,1,"Alright, Mr. Jeremy Quartering. Does your aud..."
6945,Louder_with_Crowder,2022-09-06,14,"You're running for press, you socialist cheat..."
6946,Louder_with_Crowder,2022-09-06,1,"Yeah, uh, don't get it twisted This late nigh..."
6947,Louder_with_Crowder,2022-09-07,2,It's Parody Week! You can find today's track ...


In [15]:
# j = 0
# for i in podcast_data['Transcript']:
#     if j <=5:
#         print(i)
#         print("**********************************")
#         print()
#     j +=1

In [16]:
podcast_data['Passages'] = podcast_data['Transcript'].apply(create_passages)
podcast_data['Passages']

0       [And welcome back fellow American patriots and...
1       [And welcome back fellow American patriots and...
2       [Set five years after Jedi Fallen Order, follo...
3       [Politics Without the Soap Opera with unfilter...
4       [And welcome back fellow American patriots and...
                              ...                        
6944    [Alright, Mr. Jeremy Quartering. Does your aud...
6945    [, You're running for press, you socialist che...
6946    [, Yeah, uh, don't get it twisted This late ni...
6947    [It's Parody Week! You can find today's track ...
6948    [Alright, audio listener, I will keep this bri...
Name: Passages, Length: 6949, dtype: object

In [17]:
podcast_data.to_csv('Podcast/podcast_data_processed.csv')

In [51]:
podcast_data['Passages'][4]

["And welcome back fellow American patriots and minimans standing at the ready to fight anew for our life, our liberty, and our property here at Sierra podcast on this fine Tuesday, September 13th. Daniel Horowitz back here in the house today. And folks, I was thinking has there ever been a revolution in global history where there was no opposition? It's like you're facing a revolution but you don't even know it. Where are the troops? Where's the cavalry? Where's everyone?",
 'Name me a single issue that is confronting us systemically, the transhumanism, the medical fascism, the war on food fuel, forcing us to eat bugs and drink sewage water, the border, crime, anything where there is a meaningful, coherent opposition that is likely to get into power and implement a plan or even is articulating a plan. I think back to 2010, the issues of the time were Obamacare and spending, and at least in word, there was a united front to fight them.',
 "Now they lied about it, but you can at least l

### TV Data

In [19]:
tv_data = pd.read_csv('TV/Tv_data_raw_2022.csv')
tv_data

,addeddate,contributor,description,program,tuner,text
0,2022-02-02 08:00:00,MSNBCW,"""Chris Hayes reports on some of the biggest ne...",All In With Chris Hayes,Virtual Ch. 787,"tonight on all in. >> they know it\'s true, th..."
1,2022-01-02 00:30:00,RT,"""Markets! Finance! Scandal! Keiser Report i...",Keiser Report,Channel IPTV,"""ah ah whoa, happy new year special kaiser rep..."
2,2022-01-06 13:30:00,RT,Former First Minister of Scotland Alex Salmond...,The Alex Salmond Show,Channel IPTV,"""ah, for june . welcome to the alexander. so i..."
3,2022-01-14 05:00:00,RT,"RT news, interviews and shows available as pod...",News,Channel IPTV,"""ah ah, his royal highness, no more. queen eli..."
4,2022-01-16 17:00:00,RT,"RT news, interviews and shows available as pod...",News,Channel IPTV,"""ah, with tennis superstar new york of h is de..."
...,...,...,...,...,...,...
43279,2022-05-13 03:00:00,FOXNEWSW,Greg Gutfeld examines the news of the day thro...,Gutfeld!,Virtual Ch. 760,who bought them? >> they sent them an email. >...
43280,2022-10-27 00:30:00,RT,"""RT's documentaries give a varied and unique v...",Documentary,Channel IPTV,"""have any quality for russians, all we gotta d..."
43281,2022-10-26 17:30:00,RT,"""RT's documentaries give a varied and unique v...",Documentary,Channel IPTV,"""ah ah a ah with. ringback ringback ah, becaus..."
43282,2022-10-19 05:00:00,FOXNEWSW,Powerful analysis and spirited debates with gu...,Tucker Carlson Tonight,Virtual Ch. 760,""">> they willw em. they fought for us .sn no't..."


In [20]:
tv_data['text'] = tv_data['text'].apply(pre_processing)
tv_data

,addeddate,contributor,description,program,tuner,text
0,2022-02-02 08:00:00,MSNBCW,"""Chris Hayes reports on some of the biggest ne...",All In With Chris Hayes,Virtual Ch. 787,"tonight on all in. >> they know it\'s true, th..."
1,2022-01-02 00:30:00,RT,"""Markets! Finance! Scandal! Keiser Report i...",Keiser Report,Channel IPTV,"""ah ah whoa, happy new year special kaiser rep..."
2,2022-01-06 13:30:00,RT,Former First Minister of Scotland Alex Salmond...,The Alex Salmond Show,Channel IPTV,"""ah, for june . welcome to the alexander. so i..."
3,2022-01-14 05:00:00,RT,"RT news, interviews and shows available as pod...",News,Channel IPTV,"""ah ah, his royal highness, no more. queen eli..."
4,2022-01-16 17:00:00,RT,"RT news, interviews and shows available as pod...",News,Channel IPTV,"""ah, with tennis superstar new york of h is de..."
...,...,...,...,...,...,...
43279,2022-05-13 03:00:00,FOXNEWSW,Greg Gutfeld examines the news of the day thro...,Gutfeld!,Virtual Ch. 760,who bought them? >> they sent them an email. >...
43280,2022-10-27 00:30:00,RT,"""RT's documentaries give a varied and unique v...",Documentary,Channel IPTV,"""have any quality for russians, all we gotta d..."
43281,2022-10-26 17:30:00,RT,"""RT's documentaries give a varied and unique v...",Documentary,Channel IPTV,"""ah ah a ah with. ringback ringback ah, becaus..."
43282,2022-10-19 05:00:00,FOXNEWSW,Powerful analysis and spirited debates with gu...,Tucker Carlson Tonight,Virtual Ch. 760,""">> they willw em. they fought for us .sn no't..."


In [21]:
# j = 0
# for i in tv_data['text']:
#     if j <=5:
#         print(i)
#         print("**********************************")
#         print()
#     j +=1

In [22]:
tv_data['Passages'] = tv_data['text'].apply(create_passages)
tv_data['Passages']

0        [tonight on all in. >> they know it\'s true, t...
1        ["ah ah whoa, happy new year special kaiser re...
2        ["ah, for june . welcome to the alexander. so ...
3        ["ah ah, his royal highness, no more. queen el...
4        ["ah, with tennis superstar new york of h is d...
                               ...                        
43279    [who bought them? >> they sent them an email. ...
43280    ["have any quality for russians, all we gotta ...
43281    ["ah ah a ah with. ringback ringback ah, becau...
43282    [">> they willw em. they fought for us .sn no'...
43283    ["ah a. ringback ringback because she spent wi...
Name: Passages, Length: 43284, dtype: object

In [25]:
tv_data.to_csv('TV/Tv_data_processed.csv')

### Embeddings

We specifically embed passages using a version of MPNet
that we fine-tune on the semantic text similarity (STS)
task [37], [56] using unsupervised contrastive learning for
sentence embeddings as specified in Gao et al. [57] on a
random assortment of passages from January 2022 from
our websites. We perform this fine-tuning with the default
hyperparameters (learning rate 3 × 10−5, batch size=128,
and 1M examples) specified in Gao et al. and by freezing
all but the last two layers of a public version of MPNet.2
See Appendix A for details. This ensures that our model is
attuned to the language present on our set of websites. As
seen in Table 1, despite not being trained on the SemEval
STS Benchmark [51], a benchmark for measuring the quality
of text embeddings, our model outperforms the fine-tuned
publicly released version of MPNet. After fine-tuning our
model, from the 2.1M articles, we embed 27,850,016 passages
(11.00 hours on an Nvidia RTX A6000).

In [28]:
# pip install sentence-transformers transformers

In [27]:
from sentence_transformers import SentenceTransformer, util
import torch

In [29]:
# Define the model

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [52]:
def generate_embeddings(text):
    embedding = model.encode(text, convert_to_tensor=True)
    return embedding

In [54]:
# podcast_data['Embeddings'] = podcast_data['Passages'].apply(generate_embeddings)